<table>
    <tr>
        <td>
            <img src="extra/logo_scgis.png" width="100" height="100" />
        </td>
        <td>
        <img src="extra/nansen_logo.png" width="150" height="150" />
        </td>
        <td>
            <center>
                <font size = 1>
                27-28 ОКТЯБРЯ 2017, САНКТ-ПЕТЕРБУРГ, УНИВЕРСИТЕТ ИТМО<br>
                КОНФЕРЕНЦИЯ СООБЩЕСТВА ПРИРОДООХРАННЫХ ГИС В РОССИИ<br><br>
                </font>
                <font size = 3>
                Материалы к мастер-классу<br> <b>Программирование тематических алгоритмов обработки ДДЗЗ</b><br><br>
                </font>
                <font size = 2>
                <b>Эдуард Казаков</b><br>
                Международный центр по окружающей среде и дистанционному зондированию им. Нансена
                </font>
            </center>
        </td>
    </tr>
</table>

<center>
<b>Часть 4. Полноценная задача: моделирование поля температур и поиск аномальных зон</b>
</center>